# Slew, Track and Image taking with ComCam #

This notebook is used for the level 3 integration tests from test plan LVV-P81 (https://jira.lsstcorp.org/secure/Tests.jspa#/testPlan/LVV-P81) as part of test cylce LVV-C176 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCycle/LVV-C176). The following tests are currently run as part of this notebook:

 - LVV-T2290 (https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2290)
 
Execution steps are separated by horizontal lines. Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

Last executed by E. Dennihy 20210928

***

Log onto the summit Nublado (https://summit-lsp.lsst.codes/)
Clone the GitHub repository with the notebook need for this test case using git commands.

https://github.com/lsst-ts/ts_notebooks/tree/tickets/DM-31475/procedures/MT/slew

All the following steps are executed using these notebooks.

***
Run the setup.ipnyb notebook to bring all components up and in their enabled position. Check Chronograph.

***

Bring ComCom online and tranistion it to EnabledState. Check Chronograph.

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import rubin_jupyter_utils.lab.notebook as nb
nb.utils.get_node()

In [ ]:
import os
import sys
import asyncio
import logging

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

In [ ]:
logging.basicConfig(format="%(name)s:%(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(40)

In [ ]:
await mtcs.start_task

In [ ]:
comcam = ComCam(domain=domain, log=log)

In [ ]:
comcam.set_rem_loglevel(40)

In [ ]:
await comcam.start_task

In [ ]:
await comcam.enable()

***
Slew to the next target. Choose a target such that the rotator stays within a couple of degrees of its initial position. This is because the CCW is not running (MTmount in simulation mode).

In [ ]:
target = await mtcs.find_target(el=60, az=120, mag_limit=8)
print(target)

In [ ]:
await mtcs.slew_object(target, rot_type=RotType.PhysicalSky, rot=1.9)

***
Once in position and tracking, take an image with ComCam

In [ ]:
await comcam.take_object(15)

In [ ]:
await mtcs.stop_tracking()

***
Use ComCam recent images CCS to ensure that the image was taken (http://ccs.lsst.org/RecentImages/comcam.html). 

***
Query the butler to verify that the images are there and check the metadata. This step must be verified using a separate noteboook. 

***
Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()

In [ ]:
await comcam.standby()